In [25]:
import pandas as pd
import numpy as np

users = pd.read_csv('takehome_users.csv', encoding = 'latin-1')
eng = pd.read_csv('takehome_user_engagement.csv')

In [26]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [27]:
#converting users columns to timestamp format
users['creation_time'] = pd.to_datetime(users['creation_time'])
users['last_session_creation_time'] = pd.to_datetime(users['last_session_creation_time'], unit = 's')

In [4]:
eng.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [28]:
#converting eng columns to timestamp format
eng['time_stamp'] = pd.to_datetime(eng['time_stamp'])

In [29]:
#matching name of merge column and merging, dropping users with NaT logins, using day portion of timestamp
users.columns = [x if x != 'object_id' else 'user_id' for x in users.columns]
df_merge = pd.merge(users,eng,on = 'user_id', how = 'left')
df_merge = df_merge.dropna(subset = ['time_stamp'],how = 'any')
df_merge['time_stamp'] = df_merge['time_stamp'].dt.date
df_merge.head()

,user_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,time_stamp,visited
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,2014-04-22,1.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,2013-11-15,1.0
2,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,2013-11-29,1.0
3,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,2013-12-09,1.0
4,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,2013-12-25,1.0


In [7]:
def seven_day_span(dates):
    max = 1
    count = 1
    index = 0
    for x in range(dates.index.min(),dates.index.min()+len(dates)-1):
        index = x+1
        while(np.logical_and((dates[index] - dates[x]) < pd.Timedelta('7 days'), index in range(dates.index.min(),dates.index.min()+len(dates)))): 
            count +=1
            if index < dates.index.min()+len(dates)-1:
                index +=1
            else:
                break
        if max < count:
            max = count
        count = 1
    return max

In [9]:
df_active = df_merge[['user_id','time_stamp']].reset_index().sort_values(['user_id','time_stamp']).drop('index',axis =1)
df_active.head()

,user_id,time_stamp
0,1,2014-04-22
1,2,2013-11-15
2,2,2013-11-29
3,2,2013-12-09
4,2,2013-12-25


In [10]:
df_active = df_active.groupby('user_id').agg({'time_stamp': seven_day_span})

,time_stamp
user_id,
1,1
2,3
3,1
4,1
5,1
6,1
7,1
10,7
11,1


In [15]:
df_active.columns = ['max_login_per_7']
df_active

,max_login_per_7
user_id,
1,1
2,3
3,1
4,1
5,1
6,1
7,1
10,7
11,1


In [30]:
#creating flag for active users
users = users.join(df_active,on = 'user_id')
users['active'] = users['max_login_per_7'] >=3

In [119]:
#removing irrelevant columns for model
clean_users = users.drop(['user_id','name','email'], axis = 1)
clean_users['account_life'] =  clean_users['last_session_creation_time'] -clean_users['creation_time']

In [96]:
clean_users

,creation_time,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,max_login_per_7,active,account_life
0,2014-04-22 03:53:30,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,1.0,False,0 days
1,2013-11-15 03:45:04,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,3.0,True,136 days
2,2013-03-19 23:14:52,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,1.0,False,0 days
3,2013-05-21 08:09:28,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,1.0,False,1 days
4,2013-01-17 10:14:20,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,1.0,False,5 days
5,2013-12-17 03:37:06,GUEST_INVITE,2013-12-19 03:37:06,0,0,197,11241.0,1.0,False,2 days
6,2012-12-16 13:24:32,SIGNUP,2012-12-20 13:24:32,0,1,37,NaN,1.0,False,4 days
7,2013-07-31 05:34:02,PERSONAL_PROJECTS,NaT,1,1,74,NaN,NaN,False,NaT
8,2013-11-05 04:04:24,PERSONAL_PROJECTS,NaT,0,0,302,NaN,NaN,False,NaT
9,2013-01-16 22:08:03,ORG_INVITE,2014-06-03 22:08:03,1,1,318,4143.0,7.0,True,503 days


### Creation Source

In [107]:
clean_users['creation_source'].value_counts()

ORG_INVITE            4254
GUEST_INVITE          2163
PERSONAL_PROJECTS     2111
SIGNUP                2087
SIGNUP_GOOGLE_AUTH    1385
Name: creation_source, dtype: int64

In [110]:
clean_users[clean_users['active']==1]['creation_source'].value_counts()

ORG_INVITE            553
GUEST_INVITE          360
SIGNUP                293
SIGNUP_GOOGLE_AUTH    232
PERSONAL_PROJECTS     164
Name: creation_source, dtype: int64

In [171]:
(360+1803)/len(users)

0.18025

In [111]:
clean_users[clean_users['active']==0]['creation_source'].value_counts()

ORG_INVITE            3701
PERSONAL_PROJECTS     1947
GUEST_INVITE          1803
SIGNUP                1794
SIGNUP_GOOGLE_AUTH    1153
Name: creation_source, dtype: int64

In [58]:
active_creation_ratio =  [x/len(clean_users[clean_users['active']==1]) for x in list(clean_users[clean_users['active']==1]['creation_source'].value_counts())]
active_creation_ratio

[0.34519350811485644,
 0.2247191011235955,
 0.18289637952559301,
 0.14481897627965043,
 0.10237203495630462]

In [59]:
inactive_creation_ratio = [x/len(clean_users[clean_users['active']==0]) for x in list(clean_users[clean_users['active']==0]['creation_source'].value_counts())]
inactive_creation_ratio

[0.35593383342950569,
 0.18724754760530871,
 0.17339873052510099,
 0.172533179457588,
 0.11088670898249664]

From first inspection, it seems that users are more likely to be active if they sign up for personal projects or a guest invite. I verify that this difference is signifcant with a chi-squared test

In [66]:
obs = np.array([list(clean_users[clean_users['active']==1]['creation_source'].value_counts()),list(clean_users[clean_users['active']==0]['creation_source'].value_counts())])
obs

array([[ 553,  360,  293,  232,  164],
       [3701, 1947, 1803, 1794, 1153]], dtype=int64)

In [67]:
from scipy import stats
chi2,p,dof,expected = stats.chi2_contingency(obs)
p

0.00096795414137005654

Our p-value is .000967 which is below a standard significance level of .05. We reject the null hypothesis and conclude that there is a difference in the distribution of creation sources between active and inactive users

### Session Time

Since all users without a last login time are inactive, the last session time is a useful column to determine activity. I created a new feature 'account_life' that is the length of time between account creation and last login. Active users have a much longer account life than inactive users as confirmed by the difference of means t-test.

In [120]:
clean_users[clean_users['active']==1]['account_life'].mean()

Timedelta('300 days 04:49:26.292134')

In [127]:
clean_users[clean_users['active']==0].dropna(how='any')['account_life'].mean()

Timedelta('6 days 03:25:23.686254')

In [129]:
stats.ttest_ind(clean_users[clean_users['active']==1]['account_life'].dt.days,clean_users[clean_users['active']==0].dropna(how = 'any')['account_life'].dt.days,equal_var = False)

Ttest_indResult(statistic=57.121962334340935, pvalue=0.0)

In [136]:
active_count = len(clean_users[clean_users['active']==1])
active_mean = clean_users[clean_users['active']==1]['account_life'].dt.days.mean()
active_sd = clean_users[clean_users['active']==1]['account_life'].dt.days.std()

In [137]:
inactive_count = len(clean_users[clean_users['active']==0].dropna(how='any'))
inactive_mean = clean_users[clean_users['active']==0].dropna(how='any')['account_life'].dt.days.mean()
inactive_sd = clean_users[clean_users['active']==0].dropna(how='any')['account_life'].dt.days.std()

In [139]:
diffSD = np.sqrt(active_sd**2/active_count + inactive_sd**2/inactive_count)
diffSD

5.1479037392453995

In [162]:
1-stats.norm.cdf((active_mean - inactive_mean)/diffSD)

0.0

### Mailing and Marketing

In [203]:
active_mailing = clean_users[clean_users['active']==1]['opted_in_to_mailing_list'].sum()/active_count

In [219]:
inactive_count_with_na = len(clean_users[clean_users['active']==0])
inactive_mailing = clean_users[clean_users['active']==0]['opted_in_to_mailing_list'].sum()/inactive_count_with_na

In [205]:
total_users = len(clean_users)
total_users_mailing = clean_users['opted_in_to_mailing_list'].sum()

In [206]:
SD_mailing = np.sqrt(2*(total_users_mailing/total_users) * (1-total_users_mailing/total_users) / total_users)

In [208]:
active_mailing, inactive_mailing,SD_mailing

(0.25842696629213485, 0.24812463935372187, 0.0055864381914299084)

In [209]:
(active_mailing - inactive_mailing) / SD_mailing

1.8441673541144106

In [211]:
stats.norm.cdf((active_mailing - inactive_mailing) / SD_mailing)

0.96742062277811491

We fail to reject the null hypothesis that there is a difference in proportion of users on the mailing list between active and inactive users at a .05 significance level. For a two-tailed test, the cdf should be .975 to reject the null hypothesis.

In [218]:
active_marketing = clean_users[clean_users['active']==1]['enabled_for_marketing_drip'].sum()/active_count
inactive_marketing = clean_users[clean_users['active']==0]['enabled_for_marketing_drip'].sum()/inactive_count_with_na

In [216]:
total_users_marketing = clean_users['enabled_for_marketing_drip'].sum()
SD_marketing = np.sqrt(2*total_users_marketing/total_users * (1-total_users_marketing/total_users) / total_users)

In [220]:
active_marketing, inactive_marketing, SD_marketing

(0.15355805243445692, 0.14868243893056357, 0.0046013202614193401)

In [224]:
(active_marketing - inactive_marketing) / SD_marketing

1.0596118563565047

In [225]:
stats.norm.cdf((active_marketing - inactive_marketing) / SD_marketing)

0.85533939128102965

We again fail to reject the null hypothesis at the .05 signifiance level. However, in both cases active users participate more in mailing and marketing. Even though the results do not pass the .05 significance level, there could be meaningful insight drawn from a business perspective.

### Org ID and Referral

In [232]:
active_org_id = clean_users[clean_users['active']==1][['org_id','active']].groupby('org_id').count()

In [233]:
inactive_org_id = clean_users[clean_users['active']==0][['org_id','active']].groupby('org_id').count()
inactive_org_id.columns = ['inactive']

In [238]:
org_id_table = active_org_id.join(inactive_org_id)
org_id_table['active_proportion'] = org_id_table['active'] / (org_id_table['active']+org_id_table['inactive'])

In [244]:
org_id_table.sort_values('active_proportion', ascending = False)

,active,inactive,active_proportion
org_id,,,
387,7,5,0.583333
235,6,7,0.461538
270,6,8,0.428571
399,5,8,0.384615
392,6,10,0.375000
400,3,5,0.375000
415,6,10,0.375000
117,8,14,0.363636
345,5,9,0.357143


Organization appears to be a significant factor with certain organizations sporting an active percentage as high as 58%. This makes sense as an organization could all use a platform to collaborate.

In [248]:
referred_users = clean_users[clean_users['invited_by_user_id'] > 0]
unreferred_users = clean_users[clean_users['invited_by_user_id'].isnull()]

In [259]:
active_referred = referred_users['active'].sum()/len(referred_users)
inactive_referred = unreferred_users['active'].sum()/len(unreferred_users)
total_users_referred = len(clean_users[clean_users['invited_by_user_id'].notnull()])
SD_referred = np.sqrt(2*total_users_referred/total_users * (1-total_users_referred/total_users) / total_users)

In [260]:
active_referred,inactive_referred,total_users_referred, SD_referred

(0.14227832320398939, 0.12341035285688698, 6417, 0.0064393638078617671)

In [261]:
stats.norm.cdf((active_referred-inactive_referred)/SD_referred)

0.99830572722704258

We reject the null hypothesis at a .05 signifiance level that the proportion of referred users in the active population and inactive population is the same. This shows that the referral is a useful factor in predicting active users.

In [262]:
clean_users.head()

,creation_time,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,max_login_per_7,active,account_life
0,2014-04-22 03:53:30,GUEST_INVITE,2014-04-22 03:53:30,1,0,11,10803.0,1.0,False,0 days
1,2013-11-15 03:45:04,ORG_INVITE,2014-03-31 03:45:04,0,0,1,316.0,3.0,True,136 days
2,2013-03-19 23:14:52,ORG_INVITE,2013-03-19 23:14:52,0,0,94,1525.0,1.0,False,0 days
3,2013-05-21 08:09:28,GUEST_INVITE,2013-05-22 08:09:28,0,0,1,5151.0,1.0,False,1 days
4,2013-01-17 10:14:20,GUEST_INVITE,2013-01-22 10:14:20,0,0,193,5240.0,1.0,False,5 days


In [265]:
model_data = clean_users.drop(['creation_time','last_session_creation_time', 'max_login_per_7'],axis = 1)
model_data['account_life'] = model_data['account_life'].dt.days
model_data['invited_by_user_id'] = model_data['invited_by_user_id'] > 0
model_data

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,active,account_life
0,GUEST_INVITE,1,0,11,True,False,0.0
1,ORG_INVITE,0,0,1,True,True,136.0
2,ORG_INVITE,0,0,94,True,False,0.0
3,GUEST_INVITE,0,0,1,True,False,1.0
4,GUEST_INVITE,0,0,193,True,False,5.0
5,GUEST_INVITE,0,0,197,True,False,2.0
6,SIGNUP,0,1,37,False,False,4.0
7,PERSONAL_PROJECTS,1,1,74,False,False,NaN
8,PERSONAL_PROJECTS,0,0,302,False,False,NaN
9,ORG_INVITE,1,1,318,True,True,503.0


In [267]:
df = pd.get_dummies(model_data, columns = ['creation_source','org_id'])
df.fillna(0, inplace = True)
df

,opted_in_to_mailing_list,enabled_for_marketing_drip,invited_by_user_id,active,account_life,creation_source_GUEST_INVITE,creation_source_ORG_INVITE,creation_source_PERSONAL_PROJECTS,creation_source_SIGNUP,creation_source_SIGNUP_GOOGLE_AUTH,...,org_id_407,org_id_408,org_id_409,org_id_410,org_id_411,org_id_412,org_id_413,org_id_414,org_id_415,org_id_416
0,1,0,True,False,0.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,True,True,136.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,True,False,0.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,True,False,1.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,True,False,5.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,True,False,2.0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,1,False,False,4.0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,1,1,False,False,0.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,False,False,0.0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,1,True,True,503.0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [270]:
from sklearn.model_selection import train_test_split
x = df.drop(['active'],axis = 1)
y = df['active']
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = .2,random_state = 3)

In [271]:
from sklearn import tree
from sklearn.metrics import accuracy_score
clf = tree.DecisionTreeClassifier()
clf.fit(x_train,y_train)
accuracy_score(clf.predict(x_test),y_test)

0.96750000000000003

In [272]:
accuracy_score(clf.predict(x_train),y_train)

0.99979166666666663

In [280]:
for name, importance in zip(x.columns,clf.feature_importances_):
    print(name,importance)


opted_in_to_mailing_list 0.00320261733087
enabled_for_marketing_drip 0.00662477258625
invited_by_user_id 0.00630535642634
account_life 0.868973550635
creation_source_GUEST_INVITE 0.0019758520482
creation_source_ORG_INVITE 0.00324994825916
creation_source_PERSONAL_PROJECTS 0.00322446443446
creation_source_SIGNUP 0.0019758520482
creation_source_SIGNUP_GOOGLE_AUTH 0.00464441335284
org_id_0 0.0010491063646
org_id_1 0.00144567649208
org_id_2 8.06058259257e-05
org_id_3 0.000107281788551
org_id_4 0.00017883797048
org_id_5 0.0
org_id_6 0.00122345682133
org_id_7 0.0
org_id_8 0.00108978062171
org_id_9 0.00137575027738
org_id_10 0.000794069661686
org_id_11 0.0
org_id_12 0.000689393943911
org_id_13 0.000755210959586
org_id_14 0.0
org_id_15 0.0
org_id_16 0.0
org_id_17 0.0
org_id_18 0.000491624725602
org_id_19 0.000635632482469
org_id_20 0.000786367029119
org_id_21 0.0
org_id_22 0.000383971248233
org_id_23 0.000212720811995
org_id_24 0.00111168818536
org_id_25 0.000379971547731
org_id_26 0.0
org_id_